# MIND: Modular Integration of Neural and Deductive Systems for Neuro-Symbolic AI 

This notebook demonstrates an advanced neuro-symbolic framework that integrates a neural network with symbolic reasoning.

## In-Depth Introduction

Neuro-symbolic AI is an emerging paradigm that aims to combine the pattern recognition strengths of deep learning with the explicit, rule-based reasoning capabilities of symbolic systems. The **MIND** (Modular Integration of Neural and Deductive systems) framework exemplifies this integration by allowing a neural model to learn directly from data while also obeying predefined logical constraints.

In this notebook, we illustrate the approach using a toy example where the goal is to predict a symmetric friendship relation among entities. The key idea is to enforce the symmetry rule—if entity A is a friend of entity B, then entity B must also be a friend of entity A—by adding a logic loss to the training process. This integration helps ensure that the model's predictions are consistent with domain knowledge and can lead to improved performance and interpretability.

### Why Neuro-Symbolic Integration?

- **Interpretability:** By embedding symbolic rules into the training process, the model’s decisions become more transparent and easier to explain.
- **Robustness:** The addition of logic constraints helps reduce errors that might arise solely from data-driven learning, especially when data is limited or noisy.
- **Generalizability:** This approach can be extended to more complex scenarios, enabling the development of AI systems that are both data‑driven and knowledge‑driven.

This notebook is designed for researchers, PhD students, and practitioners interested in exploring neuro‑symbolic methods and applying them to real-world AI challenges.

## Features and Usages

This updated notebook offers a range of features and demonstrates various usages of the MIND framework:

1. **Installation and Setup**
   - Easily install the required libraries (PyTorch, NumPy, Matplotlib, ipywidgets) to begin your experiments.

2. **Interactive Parameters UI**
   - Adjust key hyperparameters such as learning rate, lambda for the logic loss, number of epochs, and the probability of friendship using interactive sliders.

3. **Step-by-Step Tutorial**
   - Follow a detailed tutorial that guides you through dataset generation, model definition, training (both baseline and with logic constraints), and result visualization.

4. **Neuro-Symbolic Integration**
   - Learn how to combine neural predictions with symbolic logical constraints to enforce domain-specific rules (e.g., the symmetry of friendships).

5. **Visualization**
   - Visualize training progress, test accuracy, and the number of logical rule violations across epochs.

6. **Advanced Options**
   - Explore optional advanced configurations such as additional symbolic rules (e.g., transitivity), multi-task learning, or different model architectures.

This notebook is intended to be both a practical tutorial and a starting point for further research into neuro‑symbolic methods.

In [ ]:
# Installation cell
!pip install torch torchvision ipywidgets matplotlib numpy

# For JupyterLab users, you might need to enable the widget extension:
# jupyter nbextension enable --py widgetsnbextension

## Parameters UI

Use the interactive UI below to adjust key hyperparameters. You can change the **learning rate**, **lambda** for the logic loss, **number of epochs**, and the **friendship probability** for generating the toy dataset. The parameters will automatically update and print the current settings.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define interactive widgets for hyperparameters
learning_rate_widget = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Learning Rate:')
lambda_logic_widget = widgets.FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description='Lambda Logic:')
epochs_widget = widgets.IntSlider(value=50, min=10, max=200, step=10, description='Epochs:')
friend_prob_widget = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05, description='Friend Prob:')

display(learning_rate_widget, lambda_logic_widget, epochs_widget, friend_prob_widget)

# Store parameters in a dictionary for later use
params = {
    'learning_rate': learning_rate_widget.value,
    'lambda_logic': lambda_logic_widget.value,
    'epochs': epochs_widget.value,
    'friend_prob': friend_prob_widget.value
}

# Update parameters when sliders change
def update_params(change):
    params['learning_rate'] = learning_rate_widget.value
    params['lambda_logic'] = lambda_logic_widget.value
    params['epochs'] = epochs_widget.value
    params['friend_prob'] = friend_prob_widget.value
    print('Updated parameters:', params)

learning_rate_widget.observe(update_params, names='value')
lambda_logic_widget.observe(update_params, names='value')
epochs_widget.observe(update_params, names='value')
friend_prob_widget.observe(update_params, names='value')


## Step-by-Step Tutorial

Follow the cells below to:

1. **Generate a Toy Dataset**: Create a set of 10 entities with randomly assigned symmetric friendship relations.
2. **Define the Neural Model**: Build a neural network that predicts friendship probability from learned embeddings.
3. **Train the Baseline Model**: Train using standard binary cross-entropy loss (data only).
4. **Train the MIND Model**: Train with an additional logic loss that enforces the symmetry constraint.
5. **Visualize the Results**: Compare test accuracy and logical rule violations for both models.

In [ ]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(0)

# Define 10 entities labeled A, B, C, ... J
entities = [chr(i) for i in range(65, 75)]
num_entities = len(entities)

# Use the friend probability from the parameters UI
friend_prob = params.get('friend_prob', 0.3)
friend_matrix = np.zeros((num_entities, num_entities), dtype=int)
for i in range(num_entities):
    for j in range(i+1, num_entities):
        if np.random.rand() < friend_prob:
            friend_matrix[i, j] = 1
            friend_matrix[j, i] = 1

# No self-friendships
np.fill_diagonal(friend_matrix, 0)

# Split each unique undirected pair into training and testing (opposite directions)
train_pairs = []
train_labels = []
test_pairs = []
test_labels = []
for i in range(num_entities):
    for j in range(i+1, num_entities):
        label = friend_matrix[i, j]
        if np.random.rand() < 0.5:
            train_pairs.append((i, j)); train_labels.append(label)
            test_pairs.append((j, i)); test_labels.append(label)
        else:
            train_pairs.append((j, i)); train_labels.append(label)
            test_pairs.append((i, j)); test_labels.append(label)

train_pairs = np.array(train_pairs)
train_labels = np.array(train_labels)
test_pairs = np.array(test_pairs)
test_labels = np.array(test_labels)

print(f"Total entities: {num_entities} -> {entities}")
num_friend_pairs = int(friend_matrix.sum() / 2)
print(f"Generated {num_friend_pairs} friendship pairs (ground truth is symmetric).")

print("Example training samples:")
for idx in range(min(5, len(train_pairs))):
    i, j = train_pairs[idx]
    label = train_labels[idx]
    relation = "Friend" if label == 1 else "Not Friend"
    print(f"  {entities[i]} -> {entities[j]} : {relation}")

print("Example testing samples:")
for idx in range(min(5, len(test_pairs))):
    i, j = test_pairs[idx]
    label = test_labels[idx]
    relation = "Friend" if label == 1 else "Not Friend"
    print(f"  {entities[i]} -> {entities[j]} : {relation}")


In [ ]:
import torch
import torch.nn as nn

torch.manual_seed(0)

class FriendPredictor(nn.Module):
    def __init__(self, num_entities, embed_dim=16, hidden_dim=32):
        super(FriendPredictor, self).__init__()
        self.embedding = nn.Embedding(num_entities, embed_dim)
        self.fc1 = nn.Linear(embed_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
    
    def forward(self, i_indices, j_indices):
        emb_i = self.embedding(i_indices)
        emb_j = self.embedding(j_indices)
        concat = torch.cat([emb_i, emb_j], dim=-1)
        hidden = torch.relu(self.fc1(concat))
        logit = self.fc2(hidden).squeeze(-1)
        return logit

# Instantiate two models: baseline and MIND
num_entities = len(entities)
model_baseline = FriendPredictor(num_entities)
model_mind = FriendPredictor(num_entities)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_baseline = model_baseline.to(device)
model_mind = model_mind.to(device)


In [ ]:
# Prepare training and testing data as torch tensors
train_i = torch.tensor(train_pairs[:, 0], dtype=torch.long).to(device)
train_j = torch.tensor(train_pairs[:, 1], dtype=torch.long).to(device)
train_y = torch.tensor(train_labels, dtype=torch.float).to(device)
test_i  = torch.tensor(test_pairs[:, 0], dtype=torch.long).to(device)
test_j  = torch.tensor(test_pairs[:, 1], dtype=torch.long).to(device)
test_y  = torch.tensor(test_labels, dtype=torch.float).to(device)

# Use parameters from the UI
epochs = params.get('epochs', 50)
learning_rate = params.get('learning_rate', 0.1)

optimizer_baseline = torch.optim.Adam(model_baseline.parameters(), lr=learning_rate)
loss_fn = nn.BCEWithLogitsLoss()

baseline_acc_history = []
baseline_violations_history = []

for epoch in range(1, epochs + 1):
    model_baseline.train()
    optimizer_baseline.zero_grad()
    logits = model_baseline(train_i, train_j)
    loss = loss_fn(logits, train_y)
    loss.backward()
    optimizer_baseline.step()
    
    # Evaluate on test set
    model_baseline.eval()
    with torch.no_grad():
        test_logits = model_baseline(test_i, test_j)
        test_probs = torch.sigmoid(test_logits)
        preds = (test_probs.cpu().numpy() >= 0.5).astype(int)
        labels = test_y.cpu().numpy().astype(int)
        test_acc = (preds == labels).mean()
        baseline_acc_history.append(test_acc)
        
        # Count symmetry violations over all unique pairs
        inconsistencies = 0
        for a in range(num_entities):
            for b in range(a + 1, num_entities):
                prob_ab = torch.sigmoid(model_baseline(torch.tensor([a]).to(device), torch.tensor([b]).to(device))).item()
                prob_ba = torch.sigmoid(model_baseline(torch.tensor([b]).to(device), torch.tensor([a]).to(device))).item()
                if (prob_ab > 0.5 and prob_ba < 0.5) or (prob_ab < 0.5 and prob_ba > 0.5):
                    inconsistencies += 1
        baseline_violations_history.append(inconsistencies)

    if epoch % 10 == 0:
        print(f"[Baseline] Epoch {epoch}/{epochs} - Loss: {loss.item():.4f}, Test Acc: {test_acc*100:.1f}%, Violations: {inconsistencies}")


In [ ]:
# Train the MIND model with logic loss enforcing symmetry
epochs = params.get('epochs', 50)
learning_rate = params.get('learning_rate', 0.1)
lambda_logic = params.get('lambda_logic', 1.0)

optimizer_mind = torch.optim.Adam(model_mind.parameters(), lr=learning_rate)

mind_acc_history = []
mind_violations_history = []

for epoch in range(1, epochs + 1):
    model_mind.train()
    optimizer_mind.zero_grad()
    logits = model_mind(train_i, train_j)
    data_loss = loss_fn(logits, train_y)

    # Compute logic loss: enforce symmetry between Friend(X,Y) and Friend(Y,X) for all unique pairs
    pairs_i = []
    pairs_j = []
    for a in range(num_entities):
        for b in range(a + 1, num_entities):
            pairs_i.extend([a, b])
            pairs_j.extend([b, a])
    pairs_i = torch.tensor(pairs_i, dtype=torch.long).to(device)
    pairs_j = torch.tensor(pairs_j, dtype=torch.long).to(device)
    logits_all = model_mind(pairs_i, pairs_j)
    probs_all = torch.sigmoid(logits_all)
    # Reshape to [N_pairs, 2] so that each unique pair has two predictions
    probs_matrix = probs_all.view(-1, 2)
    diff = probs_matrix[:, 0] - probs_matrix[:, 1]
    logic_loss = torch.mean(diff * diff)

    loss = data_loss + lambda_logic * logic_loss
    loss.backward()
    optimizer_mind.step()

    model_mind.eval()
    with torch.no_grad():
        test_logits = model_mind(test_i, test_j)
        test_probs = torch.sigmoid(test_logits)
        preds = (test_probs.cpu().numpy() >= 0.5).astype(int)
        labels = test_y.cpu().numpy().astype(int)
        test_acc = (preds == labels).mean()
        mind_acc_history.append(test_acc)

        # Count symmetry violations on all unique pairs
        inconsistencies = 0
        for a in range(num_entities):
            for b in range(a + 1, num_entities):
                prob_ab = torch.sigmoid(model_mind(torch.tensor([a]).to(device), torch.tensor([b]).to(device))).item()
                prob_ba = torch.sigmoid(model_mind(torch.tensor([b]).to(device), torch.tensor([a]).to(device))).item()
                if (prob_ab > 0.5 and prob_ba < 0.5) or (prob_ab < 0.5 and prob_ba > 0.5):
                    inconsistencies += 1
        mind_violations_history.append(inconsistencies)

    if epoch % 10 == 0:
        print(f"[MIND] Epoch {epoch}/{epochs} - Loss: {loss.item():.4f}, Test Acc: {test_acc*100:.1f}%, Violations: {inconsistencies}")


In [ ]:
import matplotlib.pyplot as plt

epochs_range = range(1, params.get('epochs', 50) + 1)

plt.figure(figsize=(6, 4))
plt.plot(epochs_range, baseline_acc_history, label='Baseline', marker='o')
plt.plot(epochs_range, mind_acc_history, label='MIND', marker='s')
plt.title('Test Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim(0, 1.05)
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(6, 4))
plt.plot(epochs_range, baseline_violations_history, label='Baseline', marker='o')
plt.plot(epochs_range, mind_violations_history, label='MIND', marker='s')
plt.title('Number of Symmetry Rule Violations')
plt.xlabel('Epoch')
plt.ylabel('Violations (count)')
plt.legend()
plt.grid(True)
plt.show()

final_base_acc = baseline_acc_history[-1]
final_mind_acc = mind_acc_history[-1]
final_base_viol = baseline_violations_history[-1]
final_mind_viol = mind_violations_history[-1]

print(f"Final Baseline Accuracy: {final_base_acc*100:.1f}%")
print(f"Final MIND Accuracy: {final_mind_acc*100:.1f}%")
print(f"Final Baseline Violations: {final_base_viol}")
print(f"Final MIND Violations: {final_mind_viol}")


## Optional Advanced Options

Below are some optional advanced options for further exploration:

1. **Complex Logical Rules**: Extend the logic loss to enforce rules such as transitivity (e.g., if A is friends with B and B with C, then A should be friends with C).
2. **Multi-Task Learning**: Integrate additional tasks or relations into the model.
3. **Model Architecture Variations**: Experiment with different embedding sizes, hidden layers, or even try graph neural network architectures.
4. **Advanced Visualization**: Use techniques like t-SNE or PCA to visualize learned embeddings and the impact of logic constraints.

To activate these options, modify or add new cells to implement the desired features.

In [ ]:
# Optional: Advanced configuration example

def advanced_model_config(embed_dim=32, hidden_dim=64):
    """Return a new FriendPredictor model with advanced configuration."""
    return FriendPredictor(num_entities, embed_dim=embed_dim, hidden_dim=hidden_dim).to(device)

# Uncomment below to experiment with an advanced model configuration
# model_advanced = advanced_model_config(embed_dim=32, hidden_dim=64)
# print('Advanced model configured with embed_dim=32 and hidden_dim=64')


## Conclusion

This updated notebook has demonstrated the **MIND** framework for neuro‑symbolic AI with an integrated parameters UI, a detailed step‑by‑step tutorial, and optional advanced options. 

By combining neural network predictions with symbolic logical constraints, the MIND framework provides a robust and interpretable approach to enforcing domain-specific rules. 

Feel free to experiment with hyperparameters, explore advanced model configurations, and extend the logic constraints to suit more complex applications. Happy experimenting!